In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import matplotlib.pyplot as plt  # To visualize
import json
from collections import Counter, defaultdict

from sklearn.linear_model import LinearRegression
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 60)


In [3]:
import panphon
ft = panphon.FeatureTable()

In [4]:
import scipy.stats

In [6]:
# "../data/colex/colex_pron_wn#1.csv" 

In [55]:
df = pd.read_csv("../data/phon/wn_panphon.csv")

In [56]:
df

,SENSE_LEMMA,LANG_PRON,PRON,Conc,init_syl,init_son,init_cons,init_cont,init_delrel,init_lat,init_nas,init_strid,init_voi,init_sg,init_cg,init_ant,init_cor,init_distr,init_lab,init_hi,init_lo,init_back,init_round,init_velaric,init_tense,init_long,init_hitone,init_hireg,syl,son,cons,cont,delrel,lat,nas,strid,voi,sg,cg,ant,cor,distr,lab,hi,lo,back,round,velaric,tense,long,hitone,hireg
0,a,eng,e ɪ,2.035000,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0,0.0,1.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,0.0,0.0
1,a,hun,aː,3.561429,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,1.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.0,0.0
2,a,ita,a,3.150000,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0,0.0,1.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0
3,aaien,nld,aː i̯ ə n,4.393333,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.500000,1.000000,0.250000,0.750000,0.0,0.000000,0.250000,0.000000,1.000000,0.0,0.0,0.250000,0.250000,0.0,0.000000,0.250000,0.250000,0.250000,0.000000,0.0,0.500000,0.250000,0.0,0.0
4,aal,nld,aː l,4.585000,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.500000,1.000000,0.500000,1.000000,0.0,0.500000,0.000000,0.000000,1.000000,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.500000,0.500000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,3.765000,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,0.0,0.0,0.333333,0.833333,0.666667,0.833333,0.0,0.000000,0.166667,0.000000,0.833333,0.0,0.0,0.666667,0.666667,0.0,0.333333,0.333333,0.000000,0.333333,0.333333,0.0,0.333333,0.000000,0.0,0.0
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,4.880000,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.400000,0.600000,0.600000,0.600000,0.0,0.000000,0.200000,0.000000,0.600000,0.0,0.0,0.400000,0.200000,0.0,0.200000,0.400000,0.200000,0.200000,0.000000,0.0,0.400000,0.200000,0.0,0.0
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,2.005000,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,0.0,-1.0,0.0,0.0,0.333333,0.833333,0.500000,0.833333,0.0,0.000000,0.166667,0.000000,0.666667,0.0,0.0,0.333333,0.333333,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.0,0.166667,0.166667,0.0,0.0
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,2.775000,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,0.0,-1.0,0.0,0.0,0.400000,0.600000,0.400000,1.000000,0.0,0.000000,0.000000,0.000000,0.600000,0.0,0.0,0.400000,0.400000,0.2,0.200000,0.400000,0.000000,0.400000,0.400000,0.0,0.400000,0.400000,0.0,0.0


In [57]:
import unidecode

# Define mapping table for non-standard IPA characters
ipa_mapping = {
    "ɾ": "r",
    "ɕ": "ʃ",
    # add more mappings as needed
}

def normalize_ipa(ipa_string):
    # Convert to lowercase and remove diacritics
    ipa_string = unidecode.unidecode(ipa_string.lower())
    
    # Map any non-standard IPA characters to their standardized forms
    for key, value in ipa_mapping.items():
        ipa_string = ipa_string.replace(key, value)
    
    # Separate suprasegmental features from phonemes and remove non-valid integers
    phonemes = []
    tones = []
    for char in ipa_string:
        if char.isdigit() and int(char) in range(1, 6):
            tones.append(char)
        elif char.isalpha():
            phonemes.append(char)
    # Normalize tone markers to consistent format
    if tones:
        tones = [f"{int(tone)}" for tone in tones]
    
    # Join phonemes and tones back into a single string
    normalized_ipa = "".join(phonemes + tones)
    
    return normalized_ipa


In [58]:
df["NORM_PRON"] = df["PRON"].parallel_apply(normalize_ipa)

In [61]:
df.to_csv("../data/phon/wn_panphon_norm.csv", index=False)

In [10]:
df = df[["SENSE_LEMMA", "LANG_PRON", "PRON", "NORM_PRON", "Conc"]]

In [13]:
df

,SENSE_LEMMA,LANG_PRON,PRON,NORM_PRON,Conc
0,a,eng,e ɪ,eI,2.035000
1,a,hun,aː,a,3.561429
2,a,ita,a,a,3.150000
3,aaien,nld,aː i̯ ə n,ain,4.393333
4,aal,nld,aː l,al,4.585000
...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,urruns,3.765000
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,skima,4.880000
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,wenahs,2.005000
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,wothis,2.775000


In [25]:
df["INIT_NORM_PRON"] = df["NORM_PRON"].str[0]

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_31915/949551908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["INIT_NORM_PRON"] = df["NORM_PRON"].str[0]


In [26]:
df

,SENSE_LEMMA,LANG_PRON,PRON,NORM_PRON,Conc,INIT_NORM_PRON
0,a,eng,e ɪ,eI,2.035000,e
1,a,hun,aː,a,3.561429,a
2,a,ita,a,a,3.150000,a
3,aaien,nld,aː i̯ ə n,ain,4.393333,a
4,aal,nld,aː l,al,4.585000,a
...,...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,urruns,3.765000,u
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,skima,4.880000,s
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,wenahs,2.005000,w
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,wothis,2.775000,w


# by language

In [46]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
def get_pron_lang_non_norm(df, lang):
    df = df[df["LANG_PRON"]==lang]
    norm_prons = sorted(list(set(list(chain.from_iterable(df.PRON.tolist())))))
    X_dim = len(df)
    Y_dim = len(norm_prons)
    
    
    pron_matrix = np.zeros((X_dim, Y_dim))  
    for index, pron in enumerate(df["PRON"]):
        ls = [norm_prons.index(x) for x in list(pron)]
        for l in ls:
            pron_matrix[index][l]+=1
    
    df_pron = pd.DataFrame(pron_matrix, columns=norm_prons)
    return df_pron.corrwith(df["Conc"]).to_dict(), X_dim, Y_dim

In [47]:

for lang in list(set(df["LANG_PRON"].tolist())):
    d, x, y = get_pron_lang_non_norm(df, lang)
    v = {k:v for k,v in d.items() if v>0.3 or v<-0.3}
    if len(v)>0:
        print(lang, x,y)
        print(v)

afr 359 43
{'a': 0.6929726620365497, 'b': -0.6929726620365497, 'r': -0.9708593503364412, 'u': -0.9708593503364412, 'ŋ': 0.6929726620365497, 'ː': -0.9708593503364412, 'χ': 0.6929726620365497}
hun 2105 38
{'f': -0.34939551896935345, 'u': 0.40594944701011915}
epo 374 27
{' ': 1.0, 'a': -1.0, 'd': -1.0, 'e': -1.0, 'i': 1.0, 'k': 1.0, 'n': -1.0, 'p': -1.0, 'r': 1.0, 't': 1.0, 'u': 1.0}
ces 1708 40
{'b': 0.6272208805849364, 'k': -0.4937539796029921, 'o': 0.6691892627730949, 'r': 0.4115635572412522, 'ɲ': -0.31421004736492525, 'ʒ': 0.6272208805849364, 'ː': -0.3179839288996633}
msa 756 47
{'h': 0.5113567135797914, 'i': 0.39278753824713797, 'l': 0.31614639054265675, 'r': 0.33086924607678775, 't': -0.6616092825915599, 'ʒ': -0.5056019825425839, '͡': -0.5056019825425839}
tur 901 70
{'b': -0.7628388535167535, 'f': 0.5372818485778262, 'i': 0.5125024307290826, 'k': -0.7174286653324463, 'l': 0.5372818485778262, 'm': 0.4324555401345497, 'o': -0.312576320553134, 'y': -0.38984595015070433, 'z': 0.83979050

In [53]:
df[df["LANG_PRON"]=="tgl"]["PRON"].to_dict()

{251: 'ʔ a b n o ɾ m a l',
 255: 'ʔ a b o',
 405: 'ʔ a b u s o',
 1257: 'ʔ a ɡ ɾ i k u l t u ɾ a',
 1298: 'ʔ a h e n t e',
 1524: 'ʔ a k u s a d o',
 1549: 'ʔ a l a m a t',
 1744: 'ʔ a l k o h o l',
 1850: 'ʔ a l p a',
 1978: 'ʔ a m a i n',
 2047: 'ʔ a m b o n',
 2130: 'ʔ a m o',
 2197: 'ʔ a n a k',
 2259: 'ʔ a n a t o m i s t a',
 2389: 'ʔ a n i b e ɾ s a ɾ j o',
 2400: 'ʔ a n i m a l',
 2417: 'ʔ a n i n o',
 2578: 'ʔ a n t o k',
 2687: 'ʔ a p d o',
 2731: 'ʔ a p l a j a',
 2950: 'ʔ a ɾ a l i n',
 2954: 'ʔ a ɾ a ɾ o',
 2956: 'ʔ a ɾ a u',
 3113: 'ʔ a ɾ m a s',
 3256: 'ʔ a ɾ t e',
 3285: 'ʔ a ɾ t i s t a',
 3333: 'ʔ a s a l',
 3337: 'ʔ a s a w a',
 3432: 'ʔ a s o',
 3591: 'ʔ a s u k a l',
 3595: 'ʔ a s u p ɾ e',
 3629: 'ʔ a t e',
 4194: 'ʔ a w i t',
 4212: 'ʔ a i',
 4225: 'ʔ a j o s',
 4302: 'b a b a ʔ e',
 4317: 'b a b o i',
 4364: 'b a d i ŋ',
 4379: 'b a ɡ',
 4381: 'b a ɡ a',
 4393: 'b a ɡ a i',
 4396: 'b a ɡ i t o',
 4406: 'b a ɡ u h a n',
 4411: 'b a ɡ j o',
 4415: 'b a h a ɡ i ʔ',

In [49]:
def get_pron_lang(df, lang):
    df = df[df["LANG_PRON"]==lang]
    norm_prons = sorted(list(set(list(chain.from_iterable(df.NORM_PRON.tolist())))))
    X_dim = len(df)
    Y_dim = len(norm_prons)
    
    
    pron_matrix = np.zeros((X_dim, Y_dim))  
    for index, pron in enumerate(df["NORM_PRON"]):
        ls = [norm_prons.index(x) for x in list(pron)]
        for l in ls:
            pron_matrix[index][l]+=1
    
    df_pron = pd.DataFrame(pron_matrix, columns=norm_prons)
    return df_pron.corrwith(df["Conc"]).to_dict(), X_dim, Y_dim

In [50]:

for lang in list(set(df["LANG_PRON"].tolist())):
    d, x, y = get_pron_lang(df, lang)
    v = {k:v for k,v in d.items() if v>0.3 or v<-0.3}
    if len(v)>0:
        print(lang, x,y)
        print(v)

afr 359 27
{'G': 0.6929726620365497, 'N': 0.6929726620365497, 'a': 0.6929726620365497, 'b': -0.6929726620365497, 'h': 0.6929726620365497, 'k': 0.6929726620365497, 'r': -0.9708593503364412, 'u': -0.9708593503364412}
hun 2105 27
{'f': -0.34939551896935345, 'u': 0.40594944701011915}
epo 374 25
{'a': -1.0, 'd': -1.0, 'e': -1.0, 'i': 1.0, 'k': 1.0, 'n': -1.0, 'p': -1.0, 'r': 1.0, 't': 1.0, 'u': 1.0}
ces 1708 29
{'Z': 0.6272208805849364, 'b': 0.6272208805849364, 'k': -0.4937539796029921, 'o': 0.5135292735065909, 'r': 0.4115635572412522}
msa 756 30
{'Z': -0.5056019825425839, 'h': 0.5113567135797914, 'i': 0.39278753824713797, 'l': 0.31614639054265675, 'r': 0.33086924607678775, 't': -0.6616092825915599}
tur 901 31
{'U': 0.5622620197631572, 'a': -0.31862002111144083, 'b': -0.7628388535167535, 'f': 0.5372818485778262, 'i': 0.5125024307290826, 'k': -0.7174286653324463, 'l': 0.5372818485778262, 'm': 0.4324555401345497, 'o': -0.6104898434632688, 'r': -0.7277556538659744, 'y': -0.38984595015070433, '

# iNITIAL PRON

In [31]:
df = df.dropna(subset=["INIT_NORM_PRON"])

In [32]:
norm_prons_init = sorted(list(set((df.INIT_NORM_PRON.tolist()))))

In [33]:
X_dim = len(df)
Y_dim = len(norm_prons_init)

In [34]:
X_dim, Y_dim

(77992, 36)

In [36]:
pron_matrix = np.zeros((X_dim, Y_dim))  
for index, pron in enumerate(df["INIT_NORM_PRON"]):
    ls = [norm_prons_init.index(x) for x in list(pron)]
    for l in ls:
        pron_matrix[index][l]+=1

In [37]:
np.nonzero(pron_matrix)

(array([    0,     1,     2, ..., 77989, 77990, 77991]),
 array([14, 10, 10, ..., 32, 32, 15]))

In [38]:
df_pron = pd.DataFrame(pron_matrix, columns=norm_prons_init)

In [40]:
df_pron.corrwith(df["Conc"])

F    0.001809
I   -0.044472
N    0.006942
O    0.005569
R   -0.018704
S    0.031514
U   -0.018828
W   -0.005388
Y   -0.001055
Z    0.003619
a   -0.057397
b    0.058621
c    0.006668
d   -0.046596
e   -0.071352
f   -0.010216
g    0.039539
h    0.002861
i   -0.089178
j    0.014313
k    0.069192
l    0.026435
m    0.016915
n   -0.024475
o   -0.024303
p    0.020848
q    0.007884
r   -0.003880
s    0.005139
t    0.036603
u   -0.015201
v   -0.022135
w    0.005950
x    0.009673
y   -0.022790
z   -0.002881
dtype: float64

In [15]:
from itertools import chain

In [16]:
norm_prons = sorted(list(set(list(chain.from_iterable(df.NORM_PRON.tolist())))))

In [17]:
X_dim = len(df)
Y_dim = len(norm_prons)

In [18]:
X_dim, Y_dim

(77996, 44)

In [20]:
pron_matrix = np.zeros((X_dim, Y_dim))  
for index, pron in enumerate(df["NORM_PRON"]):
    ls = [norm_prons.index(x) for x in list(pron)]
    for l in ls:
        pron_matrix[index][l]+=1

In [21]:
np.nonzero(pron_matrix)

(array([    0,     0,     1, ..., 77995, 77995, 77995]),
 array([ 8, 22, 18, ..., 29, 31, 38]))

In [22]:
df_pron = pd.DataFrame(pron_matrix, columns=norm_prons)

In [23]:
df_pron.corrwith(df["Conc"]).to_dict()

{'1': -0.0070533366174174265,
 '2': 0.007735604747216575,
 '3': -0.009948029133716438,
 '4': 0.006249571630066299,
 '5': -0.01319178690610687,
 'E': -0.002284862455681393,
 'F': 0.005583390871535304,
 'G': -0.004993066948722249,
 'I': -0.0661045676505337,
 'N': 0.002262051845706838,
 'O': -0.002284862455681393,
 'R': -0.02861723561959621,
 'S': 0.007811610238321064,
 'U': -0.003968934316883984,
 'V': 0.0056645720879144705,
 'W': 0.012606155182018887,
 'Y': -0.0037258534609212472,
 'Z': -0.020544697995164823,
 'a': -0.024180346528743204,
 'b': 0.015792206782793736,
 'c': -0.02954883117734413,
 'd': -0.09005599644966475,
 'e': -0.14421556285515108,
 'f': -0.030003263939816678,
 'g': -0.012633620145243487,
 'h': 0.00609882229011201,
 'i': -0.1431259873856525,
 'j': -0.027498367875492678,
 'k': 0.04710494230844974,
 'l': -0.02299654382696299,
 'm': -0.05836651176704437,
 'n': -0.11477580295724528,
 'o': -0.038425728501370944,
 'p': 0.0034601550813785934,
 'q': 0.006649216174117058,
 'r': -

In [24]:
df_pron

,1,2,3,4,5,E,F,G,I,N,O,R,S,U,V,W,Y,Z,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
77992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
77994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
df = df[["SENSE_LEMMA", "LANG_PRON", "PRON", "Conc"]]

In [22]:
df["Count"] = df["PRON"].str.split().str.len()

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_25986/1896300810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Count"] = df["PRON"].str.split().str.len()


In [65]:
df

,SENSE_LEMMA,LANG_PRON,PRON,Conc,Count,Initial
0,a,eng,e ɪ,2.035000,2,e
1,a,hun,aː,3.561429,1,aː
2,a,ita,a,3.150000,1,a
3,aaien,nld,aː i̯ ə n,4.393333,4,aː
4,aal,nld,aː l,4.585000,2,aː
...,...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,3.765000,6,u
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,4.880000,5,s
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,2.005000,6,w
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,2.775000,5,w


In [112]:
def cons_voic(phone):
    r= ft.word_to_vector_list(phone, numeric=True)
    r_arr =np.array(r)[0]
    cons = r_arr[2]
    voi = r_arr[8]
    if cons==1:
        return voi
    else:
        return np.NaN
    

In [115]:
df["Init_Consonant_voi"] = df["PRON"].parallel_apply(cons_voic)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_25986/3487682184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Init_Consonant_voi"] = df["PRON"].parallel_apply(cons_voic)


In [205]:
def voiced_cons_ratio(phone):
    r= ft.word_to_vector_list(phone, numeric=True)
    r_arr =np.array(r)
    # print(r_arr)
    try:
        seg_len = r_arr.shape[0]
        # print(seg_len)
        cons = r_arr[:,2]
        voi = r_arr[:,8]
        r = r_arr[cons==1]
        r = r[r[:,8]==1]
        return r.shape[0]
    except Exception:
        return np.NaN

In [206]:
def voiceless_cons_ratio(phone):
    r= ft.word_to_vector_list(phone, numeric=True)
    r_arr =np.array(r)
    # print(r_arr)
    try:
        seg_len = r_arr.shape[0]
        # print(seg_len)
        cons = r_arr[:,2]
        voi = r_arr[:,8]
        r = r_arr[cons==1]
        r = r[r[:,8]==-1]
        # print(r.shape[0])
        return r.shape[0]
    except Exception:
        return np.NaN

In [207]:
voiceless_cons_ratio(u"s k iː m a")

2

In [208]:
voiced_cons_ratio(u"s k iː m a")

1

In [117]:
def coeff_count(df, feature):
    X_df = pd.DataFrame.from_dict(df.groupby("Conc").agg("count")[feature].to_dict(), orient="index").rename(columns={0:f"#{feature}"})
    X = X_df.index.values
    print(X.shape)
    Y = X_df[f"#{feature}"].values
    print(Y.shape)
    pr = scipy.stats.pearsonr(X,Y)
    return pr.statistic, pr.pvalue
    

In [209]:
df["Cons_voi_ratio"]=df["PRON"].parallel_apply(voiced_cons_ratio)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_25986/3533630984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cons_voi_ratio"]=df["PRON"].parallel_apply(voiced_cons_ratio)


In [210]:
df["Cons_voiceless_ratio"]=df["PRON"].parallel_apply(voiceless_cons_ratio)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_25986/2081097575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cons_voiceless_ratio"]=df["PRON"].parallel_apply(voiceless_cons_ratio)


In [199]:
df_consonat_voi = df[df["Init_Consonant_voi"]==1]

In [120]:
coeff_count(df_consonat_voi, "Init_Consonant_voi")

(3946,)
(3946,)


(0.1165062499251074, 2.1154623720035631e-13)

In [121]:
df_consonat_no_voi = df[df["Init_Consonant_voi"]==-1]
coeff_count(df_consonat_no_voi, "Init_Consonant_voi")

(4479,)
(4479,)


(0.13692545505960296, 3.429264243986782e-20)

In [110]:
cons_voic("u")

[ 1  1 -1  1 -1 -1 -1  0  1 -1 -1  0 -1  0  1  1 -1  1  1 -1  1 -1  0  0]


In [105]:
def voiceless2voice(phone):
    r = ft.word_to_vector_list(phone, numeric=True)
    r_arr = np.array(r)
    voi_arr= r_arr[:,8]
    seg = r_arr.shape[0]
    # if np.count_nonzero(voi_arr==-1)!=0:
    if np.count_nonzero(voi_arr==1):
        v2vl = np.count_nonzero(voi_arr==-1)/np.count_nonzero(voi_arr==1)
    
        return v2vl
    else:
        return np.NaN


In [94]:
voice2voiceless(u"s k iː m a")

1.5

In [ ]:
df["Voi2Voiceless"]= df["PRON"].parallel_apply(voiceless2voice)

In [102]:
def voiceless2voice(phone):
    r = ft.word_to_vector_list(phone, numeric=True)
    r_arr = np.array(r)
    voi_arr= r_arr[:,8]
    seg = r_arr.shape[0]
    # if np.count_nonzero(voi_arr==-1)!=0:
    if np.count_nonzero(voi_arr==1):
        v2vl = np.count_nonzero(voi_arr==-1)/np.count_nonzero(voi_arr==1)
    
        return v2vl
    else:
        return np.NaN


,SENSE_LEMMA,LANG_PRON,PRON,Conc,Count,Initial,Voi2Voiceless
0,a,eng,e ɪ,2.035000,2,e,0.000000
1,a,hun,aː,3.561429,1,aː,0.000000
2,a,ita,a,3.150000,1,a,0.000000
3,aaien,nld,aː i̯ ə n,4.393333,4,aː,0.000000
4,aal,nld,aː l,4.585000,2,aː,0.000000
...,...,...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,3.765000,6,u,0.200000
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,4.880000,5,s,0.666667
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,2.005000,6,w,0.500000
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,2.775000,5,w,0.666667


In [108]:
pearsonr_corr(df, "Voi2Voiceless", 0) # doesn't correlate.

(0.05509004186418571, 1.8206297992729732e-53)

In [99]:
ft.word_fts(u"e")

[<Segment [+syl, +son, -cons, +cont, -delrel, -lat, -nas, 0strid, +voi, -sg, -cg, 0ant, -cor, 0distr, -lab, -hi, -lo, -back, -round, -velaric, +tense, -long, 0hitone, 0hireg]>]

In [69]:
ft.word_fts(u"w ɛː n a h s")

[<Segment [-syl, +son, -cons, +cont, -delrel, -lat, -nas, 0strid, +voi, -sg, -cg, -ant, -cor, 0distr, +lab, +hi, -lo, +back, +round, -velaric, 0tense, -long, 0hitone, 0hireg]>,
 <Segment [+syl, +son, -cons, +cont, -delrel, -lat, -nas, 0strid, +voi, -sg, -cg, 0ant, -cor, 0distr, -lab, -hi, -lo, -back, -round, -velaric, -tense, +long, 0hitone, 0hireg]>,
 <Segment [-syl, +son, +cons, -cont, -delrel, -lat, +nas, 0strid, +voi, -sg, -cg, +ant, +cor, -distr, -lab, -hi, -lo, -back, -round, -velaric, 0tense, -long, 0hitone, 0hireg]>,
 <Segment [+syl, +son, -cons, +cont, -delrel, -lat, -nas, 0strid, +voi, -sg, -cg, 0ant, -cor, 0distr, -lab, -hi, +lo, -back, -round, -velaric, +tense, -long, 0hitone, 0hireg]>,
 <Segment [-syl, +son, +cons, +cont, -delrel, -lat, -nas, 0strid, -voi, -sg, -cg, -ant, -cor, 0distr, -lab, -hi, -lo, -back, -round, -velaric, 0tense, -long, 0hitone, 0hireg]>,
 <Segment [-syl, -son, +cons, +cont, -delrel, -lat, -nas, 0strid, -voi, -sg, -cg, +ant, +cor, -distr, -lab, -hi, -l

In [ ]:
# correlate with each other. features
# correlate with concreteness.
# with concreteness
# bilabial/plosive, [p] [b]
# voi+ cons


In [158]:
def pearsonr_corr(df, feature, conc_value):
    
    df_feature = df[["Conc", feature]].dropna()
    df_feature=df_feature[df_feature["Conc"]>=conc_value]
    X= df_feature["Conc"].values
    Y = df_feature[feature].values
    pr= scipy.stats.pearsonr(X,Y)
    return pr.statistic, pr.pvalue

In [202]:
df

,SENSE_LEMMA,LANG_PRON,PRON,Conc,Count,Initial,Voi2Voiceless,Init_Consonant_voi,Cons_voi_ratio,Cons_voiceless_ratio
0,a,eng,e ɪ,2.035000,2,e,0.000000,NaN,0.000000,0.000000
1,a,hun,aː,3.561429,1,aː,0.000000,NaN,0.000000,0.000000
2,a,ita,a,3.150000,1,a,0.000000,NaN,0.000000,0.000000
3,aaien,nld,aː i̯ ə n,4.393333,4,aː,0.000000,NaN,0.250000,0.000000
4,aal,nld,aː l,4.585000,2,aː,0.000000,NaN,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...
77991,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,3.765000,6,u,0.200000,NaN,0.500000,0.166667
77992,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,4.880000,5,s,0.666667,-1.0,0.200000,0.400000
77993,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,2.005000,6,w,0.500000,NaN,0.166667,0.333333
77994,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,2.775000,5,w,0.666667,NaN,0.000000,0.400000


In [211]:
pearsonr_corr(df, "Cons_voi_ratio", 1)

(-0.15226760855002683, 0.0)

In [212]:
pearsonr_corr(df, "Cons_voiceless_ratio", 1)

(-0.1106878108094206, 4.277582147779847e-211)

In [ ]:
# voiced bilabial plosive = b
# voiceless bilabial plosive =p

In [43]:
df["Initial"] = df["PRON"].str.split(expand=True)[0]

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_25986/673547332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Initial"] = df["PRON"].str.split(expand=True)[0]


In [63]:
df["Initial"].value_counts().to_dict()

{'k': 7439,
 'p': 6270,
 's': 6169,
 'm': 4517,
 'b': 3874,
 't': 3575,
 'a': 3300,
 'd': 3037,
 'f': 2635,
 'l': 2288,
 'v': 2023,
 'r': 1876,
 'n': 1791,
 'i': 1724,
 'ɡ': 1724,
 'h': 1425,
 'e': 1363,
 'ʃ': 1101,
 'z': 984,
 'j': 837,
 'o': 786,
 'ʋ': 741,
 'ɛ': 696,
 'ɔ': 681,
 'ʁ': 646,
 't͡ɕ': 613,
 'u': 583,
 'ʔ': 548,
 'ɑ': 540,
 't͡ʃ': 528,
 'kʰ': 528,
 'x': 515,
 'ɪ': 474,
 'd͡ʒ': 399,
 'w': 391,
 'tʰ': 383,
 'pʰ': 375,
 'ɐ': 374,
 't͡s': 322,
 'ə': 319,
 'ɦ': 317,
 'ʂ': 298,
 'ɹ': 279,
 'ɕ': 278,
 'ʒ': 264,
 'ɣ': 255,
 'sʰ': 243,
 't͡ɕʰ': 236,
 'c': 214,
 'ʊ': 189,
 'ɛ̃': 185,
 'aː': 176,
 '²': 175,
 'eː': 161,
 'y': 160,
 'ʈ͡ʂ': 151,
 'ɾ': 144,
 'ɟ': 138,
 'ɲ': 138,
 'ɑ̃': 131,
 'sˠ': 127,
 'æ': 120,
 'bˠ': 115,
 'pʲ': 109,
 'ʐ': 106,
 'bʲ': 106,
 'ʈ͡ʂʰ': 103,
 'ĩ': 99,
 'q': 98,
 'mʲ': 96,
 'oː': 95,
 'dʲ': 95,
 'ɑː': 93,
 'uː': 92,
 'θ': 89,
 'ʌ': 88,
 'tʲ': 85,
 't̪': 82,
 't̪ˠ': 79,
 'd̪': 78,
 'fʲ': 75,
 'lʲ': 75,
 'ħ': 73,
 'ɨ': 72,
 'd̪ˠ': 71,
 'iː': 68,
 'nʲ': 67,
 

In [54]:
def coeff_count(df, feature):
    X_df = pd.DataFrame.from_dict(df.groupby("Conc").agg("count")[feature].to_dict(), orient="index").rename(columns={0:f"#{feature}"})
    X = X_df.index.values
    print(X.shape)
    Y = X_df[f"#{feature}"].values
    print(Y.shape)
    pr = scipy.stats.pearsonr(X,Y)
    return pr.statistic, pr.pvalue
    

In [61]:
df_m = df[df["Initial"]=="m"]
coeff_count(df_m, "Initial")

(1633,)
(1633,)


(0.17573934559765986, 8.553000776057276e-13)

In [62]:
df_s = df[df["Initial"]=="s"]
coeff_count(df_s, "Initial")

(1923,)
(1923,)


(0.13574747984090657, 2.278216153513655e-09)

In [52]:
df_b = df[df["Initial"]=="b"]

In [55]:
coeff_count(df_b, "Initial") # more likely that concrete words are started with voiced bilabial stop.

(1487,)
(1487,)


(0.20470260721348088, 1.5664892758056369e-15)

In [57]:
coeff_count(df_p, "Initial") # not obviously

(2017,)
(2017,)


(0.10858871727187674, 1.0172714614441677e-06)

In [59]:
coeff_count(df_k, "Initial") # more concrete with k.

(2171,)
(2171,)


(0.19722458813838958, 1.7689600862066348e-20)

In [40]:
pearsonr_corr(df, "Count", 0) # more concrete, fewer phonemes.

(-0.23006247884997993, 0.0)

In [41]:
pearsonr_corr(df, "Count", 3) # more concrete, fewer phonemes.

(-0.11078811077490539, 5.454252634406433e-133)